[Node 64: Programme der Python Standard Library](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node64.html)

Navigation:

**Next:** [Verteilte Programme mit XML-RPC](node65.ipynb) **Up:** [Netzwerk Programmierung](node62.ipynb) **Previous:** [Überblick Netzwerkprogrammierung](node63.ipynb)

##  Module der Python-Standardbibliothek

Klassische Anwendungen wie FTP, Versand und Empfangen von E-Mails oder Telnet können synchron abgearbeitet werden, d.h. eine Anforderung wird an einen Server gesendet, auf dessen Antwort gewartet und dann weitergearbeitet. Für zahlreiche Netzwerkdienste existieren Module in der Python-Standardbibliothek wie z.B.  ``ftplib, poplib, imaplib, smtplib, nntplib, gopherlib, telnetlib`` .  

### Server mit SocketServer

Wir gehen hier nicht auf die low-level Berkley-Socket-API ein, sondern beschränken uns auf die Klassen des  ``SocketServer`` -Moduls, die die Komplexität der low-level Socket-API verbergen. 

Bei einem ``SocketServer``  geht man wie folgt vor:  
* Leite Verarbeitungsklasse von  ``BaseRequestHandler``  ab und  überschreibe die  ``handle``  Methode 
* Benutze Serverklasse  ``TCPServer, UDPServer``  und instanziiere diese mit Server-Adresse und Verarbeitungsklasse
* Starte Ereignisschleife des Servers.

Ein Beispiel für einen einfachen Server auf TCP-Basis:

In [ ]:
from SocketServer import TCPServer, BaseRequestHandler

class HelloHandler(BaseRequestHandler):
    def handle(self):
        print "Serving client:", self.client_address
        self.request.sendall('Hello Client! I am a HelloHandler\r\n')
        
TCPServer.allow_reuse_address = True
srv = TCPServer(("", 7070), HelloHandler)
srv.serve_forever()

Man führt dieses Programm auf dem Server aus und kontaktiert das Server-Programm von einem Client auf der Shell-Kommando-Zeile mit ``nc (netcat)``:

```bash
$ nc localhost 7070
Hello Client! I am a HelloHandler
```

Auf dem Server erhält man bei dieser Verbindung eine Meldung:
<pre>
Serving client: ('127.0.0.1', 48908)
</pre>
(siehe oben...)

---
Ein Echo-Server sieht folgendermaßen aus:

In [ ]:
from SocketServer import TCPServer, StreamRequestHandler

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print "Serving client:", self.client_address
        for line in (self.rfile):
            self.wfile.write("S:" + line)
            
TCPServer.allow_reuse_address = True
srv = TCPServer(("", 7071), EchoHandler)
srv.serve_forever()

Startet man obigen Server, so erhält man auf dem Klienten folgende Ein- und Ausgabe:
<pre>
$ nc localhost 7071
Hallo
S:Hallo
Echo !
S:Echo !
</pre>

Der Nachteil des gerade vorgestellten Echo-Servers liegt in der Tatsache, dass er nur einen Klienten zur gleichen Zeit bedienen kann, da der Server nur mit einem Thread läuft. 
Als Lösung bietet sich ein asynchroner Server (wie `Twisted`) an, oder man führt die  ``handle``-Methode in einem eigenen Prozess oder Thread aus. 

Im folgenden Beispiel wird  ``ThreadingTCPServer``  anstatt  ``TCPServer``  verwendet:

In [ ]:
from SocketServer import ThreadingTCPServer, StreamRequestHandler

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print "Serving client:", self.client_address
        for line in (self.rfile):
            self.wfile.write("S:" + line)
            
ThreadingTCPServer.allow_reuse_address = True
srv = ThreadingTCPServer(("", 7072), EchoHandler)
srv.serve_forever()

### E-Mails senden mit `smtplib.SMTP`

Um E-Mails synchron zu senden, kann man die Klasse  ``SMTP``  aus dem  ``smtplib``-Modul verwenden. In folgendem Programm wird demonstriert, wie man eine E-Mail mit dem  ``email.message``-Modul zunächst komponiert und sich dann am Mailserver der LMU-Physik authentifiziert und die E-Mail über das SMTP-Protokoll (Simple Mail Transfer Protocol) an seine eigene Adresse versendet: 
[``test_send_mail.py``](source/test_send_mail.py) 

Das Passwort wird über die Funktion ``getpass`` zur Laufzeit des Programms eingelesen.

Das SMTP stammt ursprünglich aus dem Jahr 1982 und ist sehr simpel, aber immer noch die Grundlage von allem Mailverkehr im Internet. Es sieht keine Authentifizierung vor. Jeder Server kann im Prinzip an jeden anderen Server Mails schicken, ohne beweisen zu müssen, dass er dazu berechtigt ist, oder dass die Mails, die er weiterleitet, tatsächlich von den behaupteten Absendern stammen. Diese Eigenheit im Design des Protokolls ist zwar im Geiste eines freien und offenen Internets, führte aber auch zu dem riesigen Problem mit Spam-Mails, das wir heute haben.

### E-Mails lesen mit `imaplib.IMAP4`

Über die  ``IMAP4_SSL``-Funktion des  ``imaplib``-Moduls können E-Mails von einem Server gelesen werden: [``imaplib_imap4client.py``](source/imaplib_imap4client.py)